# Set up `rxmx.walker.Walker`

In [13]:
import lzma
from pathlib import Path

import dill as pickle
import numpy as np
from tqdm import tqdm

In [14]:
from scipy import stats
from scipy.optimize import minimize

In [15]:
import elm

In [16]:
import rxmc

In [17]:
evidence_dir = Path("../evidence/data")
model_dir = Path("../model/")
likelihood_model_dir = Path("../likelihood_model/")
prior_path = Path("../prior/chuq_prior.ipynb")

## Prior

In [19]:
from collections import OrderedDict

prior_mean_chuq = OrderedDict(
    [
        ("V0", 56.19),
        ("Ve", -0.36),
        ("Vt", 13.82),
        ("r0", 1.20),
        ("r0_0", -0.20),
        ("a0", 0.73),
        ("Wv0", 9.92),
        ("Wve0", 33.15),
        ("Wvew", 24.00),
        ("rw", 1.32),
        ("rw_0", -0.41),
        ("aw", 0.69),
        ("Ws0", 10.59),
        ("Wst", 27.09),
        ("Wse0", 20.00),
        ("Wsew", 36.38),
        ("Vso", 5.58),
        ("rso", 1.29),
        ("rso_0", -1.12),
        ("aso", 0.61),
        ("rc", 1.25),
        ("rc_0", 0.13),
    ]
)

prior_std_chuq = OrderedDict(
    [
        ("V0", 1.625),  # (1.43 + 1.82) / 2
        ("Ve", 0.025),  # (0.03 + 0.02) / 2
        ("Vt", 6.14),  # (7.03 + 5.25) / 2
        ("r0", 0.03),
        ("r0_0", 0.125),
        ("a0", 0.025),
        ("Wv0", 3.775),
        ("Wve0", 22.425),
        ("Wvew", 10.42),
        ("rw", 0.08),
        ("rw_0", 0.34),
        ("aw", 0.05),
        ("Ws0", 3.69),
        ("Wst", 10.50),
        ("Wse0", 21.26),
        ("Wsew", 18.71),
        ("Vso", 0.55),
        ("rso", 0.11),
        ("rso_0", 0.48),
        ("aso", 0.04),
        ("rc", 0.12),
        ("rc_0", 0.105),
    ]
)

In [20]:
covariance = np.diag(list(prior_std_chuq.values())) ** 2
mean = np.array(list(prior_mean_chuq.values()))

In [21]:
prior_distribution = stats.multivariate_normal(mean, covariance)

## read in observations

In [22]:
with lzma.open(evidence_dir / "nn_elastic_observations.xz", "rb") as f:
    nn_elastic_obs = pickle.load(f)
with lzma.open(evidence_dir / "pp_elastic_observations.xz", "rb") as f:
    pp_elastic_obs = pickle.load(f)

## read in physical models

In [23]:
with open(model_dir / "chuq_nn_model.pkl", "rb") as f:
    nn_model = pickle.load(f)

In [24]:
with open(model_dir / "chuq_pp_model.pkl", "rb") as f:
    pp_model = pickle.load(f)

## read in likelihood models

In [25]:
with open(likelihood_model_dir / "likelihood_model.pkl", "rb") as f:
    likelihood_model = pickle.load(f)

## Set up Evidence object

In [26]:
pp_constraints = [
    rxmc.constraint.Constraint(
        observations=obs,
        physical_model=pp_model,
        likelihood_model=likelihood_model,
    )
    for entry_id, obs in pp_elastic_obs.items()
]
nn_constraints = [
    rxmc.constraint.Constraint(
        observations=obs,
        physical_model=nn_model,
        likelihood_model=likelihood_model,
    )
    for entry_id, obs in nn_elastic_obs.items()
]

In [27]:
evidence = rxmc.evidence.Evidence(constraints=nn_constraints + pp_constraints)

In [28]:
import jitr

In [29]:
starting_loc = np.mean(
    jitr.optical_potentials.chuq.get_samples_democratic((1, 1)), axis=0
)
starting_loc

array([56.08949183, -0.35849618, 13.93415718,  1.19718841, -0.19497066,
        0.73447478, 10.55611553, 34.3861483 , 24.90766928,  1.32706245,
       -0.39752667,  0.69519624, 10.91565673, 28.85849212, 19.36201193,
       41.07509524,  5.53077466,  1.30125197, -1.12924722,  0.60825946,
        1.24718711,  0.12861764])

In [30]:
rng = np.random.default_rng(13)

In [31]:
proposal = rxmc.proposal.NormalProposalDistribution(
    cov=np.diag((np.diag(prior_distribution.cov) * 0.01**2)),
)

In [35]:
sampling_config = rxmc.param_sampling.SamplingConfig(
    starting_location=starting_loc,
    params=elm.chuq.params,
    proposal=proposal,
    prior=prior_distribution,
)

In [36]:
walker = rxmc.walker.Walker(
    model_sample_conf=sampling_config,
    evidence=evidence,
    rng=rng,
)

In [39]:
with lzma.open("./walker_chuq.pkl.xz", "wb") as f:
    pickle.dump(walker, f)

In [42]:
walker.walk(10)

Batch: 1/1 completed, 100 steps. 
  Model parameter acceptance fraction: 0.460


0.46

In [43]:
%%time
walker.walk(100)

Batch: 1/1 completed, 100 steps. 
  Model parameter acceptance fraction: 0.490
CPU times: user 59.3 s, sys: 19.7 ms, total: 59.3 s
Wall time: 51.2 s


0.49